In [6]:
%pip install pandas
%pip install plotly
%pip install scipy
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Visualizing cone calorimeter data
Note: for best results, view this notebook in VSCode with the Jupyter extension installed, not JupyterLab.

In [7]:
import pandas as pd
import scipy

import json
from pathlib import Path
from datetime import date

import ipywidgets as widgets
from IPython.display import display, clear_output

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [8]:
def colorize(text, color):
    if color == "red":
        color = 196
    elif color == "green":
        color = 45
    elif color == "blue":
        color = 80
    elif color == "yellow":
        color = 226
    elif color == "purple":
        color = 92
    elif color == "cyan":
        color = 123
    elif color == "white":
        color = 15
    elif color == "black":
        color = 0
    else:
        color = 15

    return f"\x1b[38;5;{color}m{text}\x1b[0m"

In [9]:
input_path = Path("../OUTPUT/")

source_widget = widgets.Dropdown(options=[x.name for x in input_path.iterdir() if x.is_dir()], description="Source:")
display(source_widget)

Dropdown(description='Source:', options=('FAA', 'FTT', 'MIDAS'), value='FAA')

In [10]:
metadata_files = list(Path(input_path / Path(source_widget.value)).glob("**/*.json"))

# get test metadata
for file in metadata_files:
        metadata = json.load(file.open())
        # display relevant info about each test

        print(colorize(f"Test {file.stem.replace("_metadata", "")}:", "blue"))
        print(f" - Date: {metadata.get("date")}")
        print(f" - Heat flux (kw/m^2): {metadata.get('heat_flux_kw/m^2')}")
        print(f" - Material: {metadata.get('material_name')}")
        print(f" - Specimen description: {metadata.get("specimen_description")}")
        print(f" - General comments: {metadata.get('comments')}")

Test 021204A6:
 - Date: 2002-12-04T04:38:00
 - Heat flux (kw/m^2): 50.0
 - Material: None
 - Specimen description: None
 - General comments: None
Test 021210A1:
 - Date: 2002-12-10T02:14:00
 - Heat flux (kw/m^2): 65.0
 - Material: None
 - Specimen description: None
 - General comments: None
Test 021210A2:
 - Date: 2002-12-10T02:36:00
 - Heat flux (kw/m^2): 65.0
 - Material: None
 - Specimen description: None
 - General comments: None
Test 021210A3:
 - Date: 2002-12-10T02:56:00
 - Heat flux (kw/m^2): 65.0
 - Material: None
 - Specimen description: None
 - General comments: None
Test 021210A4:
 - Date: 2002-12-10T03:21:00
 - Heat flux (kw/m^2): 65.0
 - Material: None
 - Specimen description: None
 - General comments: None
Test 021210A5:
 - Date: 2002-12-10T03:50:00
 - Heat flux (kw/m^2): 65.0
 - Material: None
 - Specimen description: None
 - General comments: None
Test 021219A1:
 - Date: 2002-12-19T02:10:00
 - Heat flux (kw/m^2): 35.0
 - Material: None
 - Specimen description: None
 - G

In [101]:
filename_widget = widgets.Dropdown(options=[str(x.stem).replace("_metadata", "") for x in metadata_files], description="Filename:")

data_file_path = Path(filename_widget.value + "_data.csv")
data = pd.read_csv(input_path / Path(source_widget.value) / data_file_path)

# Select X & Y axis
x_axis_widget = widgets.Dropdown(description='X axis:', options=data.columns)
y_axis_widget = widgets.Dropdown(description='Y axis:', options=data.columns)
sec_y_axis_widget = widgets.Dropdown(description='Sec. Y axis:', options=data.columns)
sec_x_axis_widget = widgets.Dropdown(description='Sec. X axis:', options=data.columns)

fig = make_subplots(specs=[[{"secondary_y": True}]])

def update_data(event=None):

    data_file_path = Path(filename_widget.value + "_data.csv")
    data = pd.read_csv(input_path / Path(source_widget.value) / data_file_path)

    x_axis_widget.options = data.columns
    y_axis_widget.options = data.columns

    sec_y_axis_widget.options = data.columns
    sec_x_axis_widget.options = data.columns

    plot()

def plot(event=None):

    clear_output()

    primary_box = widgets.Box(children=[x_axis_widget, y_axis_widget])
    secondary_box = widgets.Box(children=[sec_x_axis_widget, sec_y_axis_widget])
    vbox = widgets.VBox([filename_widget, primary_box, secondary_box])

    display(vbox)

    y_data = data[y_axis_widget.value]
    x_data = data[x_axis_widget.value]

    sec_y_data = data[sec_y_axis_widget.value]
    sec_x_data = data[sec_x_axis_widget.value]

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if y_axis_widget.value != x_axis_widget.value:
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines', name=f"{y_axis_widget.value} vs {x_axis_widget.value}"), secondary_y=False)
    
    if sec_y_axis_widget.value != sec_x_axis_widget.value:
        fig.add_trace(go.Scatter(x=sec_x_data, y=sec_y_data, mode='lines', name=f"{sec_y_axis_widget.value} vs {sec_x_axis_widget.value}"), secondary_y=True)
    

    fig.show()

# TODO: also allow setting the scale for each axis

x_axis_widget.observe(plot, names="value")
y_axis_widget.observe(plot, names="value")
sec_x_axis_widget.observe(plot, names="value")
sec_y_axis_widget.observe(plot, names="value")
filename_widget.observe(update_data, names="value")

plot()


In [12]:
# print out material info again:
metadata = json.load((input_path / Path(source_widget.value) / Path(filename_widget.value + "_metadata.json")).open())

test_name = filename_widget.value

print(colorize(f"Selected test: {test_name}", "blue"))
print(f" - Date: {metadata.get('date')}")
print(f" - Heat flux (kw/m^2): {metadata.get('heat_flux_kw/m^2')}")
print(f" - Material: {metadata.get('material_name')}")
print(f" - Specimen description: {metadata.get("specimen_description")}")
print(f" - General comments: {metadata.get('comments')}")

Selected test: 021204A6
 - Date: 2002-12-04T04:38:00
 - Heat flux (kw/m^2): 50.0
 - Material: None
 - Specimen description: None
 - General comments: None
